# Approximate the MST Weight
Source: http://www.cs.princeton.edu/courses/archive/spring04/cos598B/bib/Czumaj-Sohler-EstimMST.pdf

In the paper:

$P$ is a metric space (In the code, $(P,d)$ is a metric space)  we have that: $\left|P\right| = n$

$G^{(i)}$ and is the $(1+\epsilon)^i$-radius graph on $(P,d)$

The distances are rescaled in $O(n)$ time so that $d(x,y) \in [0,(1+\epsilon)^r]$ where $(1+\epsilon)^r$ is the smallest power of $1+\epsilon$ above $\frac{4n}{\epsilon}$

In [1]:
import numpy as np
from scipy.stats import rv_discrete 
from scipy.spatial import distance
from math import *
import itertools
import sys

<img src="degree_estimate.png">

In [2]:
def Degree_Estimate(P,d,threshold,p,constant=1):
    n = len(P)
    clogn = constant*log(n)
    l = 1
    while True:
        l = 2*l
        N = clogn*l
        S = np.random.choice(n,N,replace=True)
        #number of vertices in S that are adjacent to p in G_{threshold}
        Y = sum(1 for elt in S if (d(p,elt) < threshold))
        if( Y > clogn or l > n):
            return float(Y*n)/float(N)

<img src ="clique_tree_traversal.png">

In [3]:
#This is Clique_Tree_Traversal from the paper modified to have the stopping criteria outlined in the paper.
#threshold = (1+eps)^i
def Modified_Clique_Tree_Traversal(P,d,p,X,threshold,epsilon,r):
    R = [p] #representive vertices
    E = [p] #explored vertices
    U = range(0,p)+range(p+1,len(P)) #unexplored vertices, everything but p
    teps = threshold*epsilon #this is rhl being a C programmer and saving a multiply
    edges = [(d(a,b),a,b) for a in R for b in U]
    #todo remove edges of weight larger than threshold*(1+epsilon)
    edges.sort(reverse=True)
    for i in xrange(len(edges) - 1, -1, -1):
            if edges[i][0] >= threshold*(1+epsilon):
                del edges[i]
    while True:
        if (len(edges) == 0 or len(E) > X or len(R) > (4*r/epsilon)):
            return 0
        #Since all the unexplored vertices are now explored
        #We know that we have explored all of the vertices connected to p in the threshold
        #perhaps we need to keep track of connected components with a d.s. datastructure?
        if (len(U) == 0): 
            return 1
        w = edges[-1][0] #the minimum weight edge between the representative and unexplored vertices
        a = edges[-1][1] #the representative vertex
        b = edges[-1][2] #the unexplored vertex
        E.append( b) #mark b as explored
        U.remove( b) #remove b from the unexplored
        if( d(a,b) > teps): #if d(a,b) > teps
            R.append(b) #append the now explored vertex b to the representative vertex list
            edges.extend( [(d(b,h),b,h) for h in U]) #compute distance from b to the remaining U guys.
            edges.sort(reverse=True) #merge the two sorted lists
        #remove all edges to b in the edges list
        for i in xrange(len(edges) - 1, -1, -1):
            if (edges[i][2] == b) or (edges[i][0] >= (threshold*(1+epsilon))):
                del edges[i]

<img src="number_of_connected_components.png">

In [4]:
#threshold = (1+eps)^i
def Number_Of_Connected_Components(P,d,threshold,T,epsilon,r):
    n = len(P)
    #Build the distribution P[X >= k] = 1/k$
    probabilities = np.array([1.0/k - 1.0/(k+1) for k in xrange(1,n+1)])
    distrib = rv_discrete(values=((xrange(1,n+1)),probabilities))
    s = 0
    chat=0
    while (s <= T):
        s = s+1
        p = np.random.randint(n) #draw a vertex uniformly, independently at random.
        X = distrib.rvs(size=1)
        D = Degree_Estimate(P,d,threshold,p)
        if (D < 2*X):
            chat = chat + Modified_Clique_Tree_Traversal(P,d,p,X,threshold,epsilon,r)
    return (float(n)/s)*chat

<img src="metric_mst_approx.png">

In [5]:
def Metric_MST_Approximation(P,orig_d,epsilon,constant=1):
    n = len(P)
    #the paper asks for this.. for now i've turned it off
    #epsilon = epsilon/4.0
    r = int(ceil(log(4*n/epsilon)/log1p(epsilon)))
    vtx = np.random.randint(n) #draw a vertex uniformly, independently at random.
    #by the triangle inequality d(x,y) <= 2*max_d_to_vtx
    max_d_to_vtx = max([orig_d(P[x,:],P[vtx,:]) for x in xrange(0,n)])
    #multiplying all the distances by this makes them in the range [0,(1+epsilon)^r]
    rescale_factor = pow(1+epsilon,r)/(2*max_d_to_vtx)
    #we turn the original distance into the rescaled distance
    d = lambda x,y: orig_d(P[x,:],P[y,:])*rescale_factor    
    c = 0
    T = int(constant*n/pow(epsilon,6.0))
    for i in xrange(0,r):
        threshold = pow((1+epsilon),i)
        print threshold
        sys.stdout.flush()
        cc = Number_Of_Connected_Components(P,d,threshold,T,epsilon,r)
        print "cc =", cc
        c = c + threshold*cc
    return (n - pow(1+epsilon,r)) + epsilon*c

In [6]:
P = np.random.random((10,5))

In [7]:
print distance.euclidean(P[7,:],P[4,:]) #we just need to be able to do stuff like this

1.3670356698


/** This is currently totally wrong, the estimates on the # of connected components are way off **/

In [8]:
#print Metric_MST_Approximation(P,distance.euclidean,.4)